# Imports and definitions

In [1]:
from XRDXRFutils.notebook_utils import *
from os.path import basename

#path_shared = '/home/shared/'
path_shared = '../../shared/'

path_database = path_shared + 'Database16_42_da_sistemare/'   # tabulated phases

# Read phase database

In [2]:
filenames = sorted(glob(path_database + '/*.cif'))
if not filenames:
    warnings.warn('No files found')

In [3]:
len(filenames)

1337

# Check phases

In [4]:
filenames_to_fix = []

for filename in filenames:
    phase = Phase(name = filename)
    with open(filename, 'r') as f:
        for line in f:
            x = line.split()
            if x:
                y = x[0]

                if y in ['_chemical_formula_sum', '_chemical_name_mineral', '_chemical_name_common']:
                    value = ' '.join(x[1:]).replace("'", '')
                    value = value.strip()
                    if value != '':
                        phase[y] = value

                if y == 'name':
                    phase[y] = ' '.join(x[1:]).replace("'", '')

                if y == 'point':
                    phase[y] = int(x[1])

                if y == '_pd_peak_intensity':
                    z = loadtxt(f, unpack = True, dtype = float)
                    phase[y] = z
    #if ('_chemical_formula_sum' not in phase.keys()):
    #if ('_chemical_name_mineral' not in phase.keys()):
#    if ( # If one of the two names exists and is the same as chemical formula, or if no name exists
#        ( ('_chemical_name_common' in phase.keys()) and ('_chemical_formula_sum' in phase.keys()) and (phase['_chemical_name_common'] == phase['_chemical_formula_sum']) ) or
#        ( ('_chemical_name_mineral' in phase.keys()) and ('_chemical_formula_sum' in phase.keys()) and (phase['_chemical_name_mineral'] == phase['_chemical_formula_sum']) ) or
#        ( ('_chemical_name_mineral' not in phase.keys()) and ('_chemical_name_common' not in phase.keys()) )
#    ):
    if ( # If name is a permutation of chemical formula
        ('_chemical_name_common' in phase.keys()) and ('_chemical_formula_sum' in phase.keys()) and
        (len(phase['_chemical_name_common']) == len(phase['_chemical_formula_sum'])) and (sorted(phase['_chemical_name_common']) == sorted(phase['_chemical_formula_sum']))
    ):
        filenames_to_fix.append(basename(filename))

In [5]:
print(len(filenames_to_fix))
for f in filenames_to_fix:
    print(f)

0


### View phase database

In [6]:
print('Reading phases from: ' + path_database)
database = DatabaseXRD().read_cifs(path_database)
print(f'Total number of phases: {len(database)}')

df_phases = DataFrame(columns = ['Name in database', 'Chemical formula', 'N. of samples'])
for phase_name, phase_set in sorted(database.items()):
    p = phase_set[0]
    if '_chemical_formula_sum' in p:
        formula = p['_chemical_formula_sum']
    else:
        formula = None
    df_phases.loc[len(df_phases)] = [phase_name, formula, len(phase_set)]
elements_excluded = []
df_phases

Reading phases from: ../../shared/Database16_42_da_sistemare/
Total number of phases: 401


,Name in database,Chemical formula,N. of samples
0,((H2 N)2 (Fe (C O)3)2),C6 H4 Fe2 N2 O6,1
1,(N H4)4 (Fe (C N)6) (H2 O)1.5,C6 H19 Fe N10 O1.5,1
2,(Pb O) (Pb (Cr O4)),Cr O5 Pb2,1
3,4-Ferrocenylmethylene-2-phenyl-4H-oxazol-5-one,C20 H15 Fe N O2,1
4,Akaganeite,Cl0.675 Fe4 O8,2
...,...,...,...
396,magnesium sulfate,Mg O4 S,4
397,shattuckite,Cu5 H2 O14 Si4,1
398,zinc_oxide,O Zn,5
399,"zincian paratacamite, syn",Cl2 Cu3.58 H6 O6 Zn0.42,2


### View phases with same chemical formula and multiple names

In [7]:
df_stats = df_phases.groupby('Chemical formula')[['Name in database']].count()
df_stats = df_stats[df_stats['Name in database'] > 1].reset_index()
df_stats

,Chemical formula,Name in database
0,Al,2
1,Al H3 O3,6
2,Al2 Co O4,3
3,Al2 H4 O9 Si2,3
4,Al2 O3,7
...,...,...
58,O2 Si,9
59,O2 Ti,3
60,O4 Pb S,2
61,S Zn,8


In [11]:
df_phases[df_phases['Chemical formula'] == 'O Zn']
#df_phases[df_phases['Chemical formula'] == 'Al H3 O3']

,Name in database,Chemical formula,N. of samples
378,Zinc oxide,O Zn,3
380,Zincite,O Zn,8
398,zinc_oxide,O Zn,5
400,zincite,O Zn,1


### View phases with same name and multiple chemical formulas

In [10]:
df_stats = df_phases.groupby('Name in database')[['Chemical formula']].count()
df_stats = df_stats[df_stats['Chemical formula'] > 1].reset_index()
df_stats

,Name in database,Chemical formula
